In [1]:
import pandas as pd
import numpy as np
#import pandasql as ps
from datetime import datetime, timedelta
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from pyhive import presto
from pymongo import MongoClient
import warnings
warnings.filterwarnings("ignore")

presto_conn = presto.connect(
    host='presto.processing.yoda.run',
    port=80,
    protocol='http',
    catalog='hive',
    username='mayank.jha@rapido.bike',
    # requests_kwargs=req_kw,
)

In [4]:
pd.to_datetime('2021-06-27').strftime('%Y-%V')

'2021-25'

In [3]:
rides_query = """SELECT rider,
                    serviceobj_service as service,
                    serviceobj_city,
                    COUNT(DISTINCT(_id)) as all_service_rides,
                    coalesce(sum(case when feedback_customerrating = -1 then 0 else feedback_customerrating end)/(nullif(sum(case when feedback_customerrating = -1 then 0 else 1 end),0)*1.0),0) as rating
                from legacy.orders
                where orderdate >= '2021-07-05'
                    and orderdate <= '2021-07-11'
                    and serviceobj_servicetype <> 'auto'
                    and status = 'dropped'
                    and spdfraud_flag != 1
                    and serviceobj_city in ('Bangalore','Hyderabad','Delhi')
                    group by 1, 2, 3 """

df_net_captains = pd.read_sql(rides_query, presto_conn)

df_net_captains.to_csv('rides.csv',index=False)

In [3]:
#Start time and end time declaration
CC = [
    ['2021-06-28','2021-07-04']
]

In [6]:
# Find out the total rides in a week and average rating. Choose the correct ratings formula from the query

rides_data = pd.DataFrame()

for i in CC:
    rides_query = """SELECT rider,
                    CAST(DATE_TRUNC('week',CAST(orderdate AS DATE)) as varchar) as order_week,
                    -- serviceobj_city as city,
                    COUNT(DISTINCT(orderdate)) as wdays,
                    COUNT(DISTINCT(_id)) as all_service_rides,
                    -- avg(case when feedback_customerrating = -1 then 5 else feedback_customerrating end) as rating
                    coalesce(sum(case when feedback_customerrating = -1 then 0 else feedback_customerrating end)/(nullif(sum(case when feedback_customerrating = -1 then 0 else 1 end),0)*1.0),0) as rating
                    from legacy.orders
                    where orderdate >= '{sd}'
                    and orderdate <= '{ed}'
                    and serviceobj_servicetype <> 'auto'
                    and status = 'dropped'
                    and spdfraud_flag != 1
                    and serviceobj_city = 'Hyderabad'
                    group by 1, 2""".format(sd = i[0] , ed = i[1])

    print(rides_query)
    df_net_captains = pd.read_sql(rides_query, presto_conn)
    rides_data = pd.concat([rides_data,df_net_captains])
    print(df_net_captains.head(5))
    print(len(rides_data))

SELECT rider,
                    CAST(DATE_TRUNC('week',CAST(orderdate AS DATE)) as varchar) as order_week,
                    -- serviceobj_city as city,
                    COUNT(DISTINCT(orderdate)) as wdays,
                    COUNT(DISTINCT(_id)) as all_service_rides,
                    -- avg(case when feedback_customerrating = -1 then 5 else feedback_customerrating end) as rating
                    coalesce(sum(case when feedback_customerrating = -1 then 0 else feedback_customerrating end)/(nullif(sum(case when feedback_customerrating = -1 then 0 else 1 end),0)*1.0),0) as rating
                    from legacy.orders
                    where orderdate >= '2021-06-28'
                    and orderdate <= '2021-07-04'
                    and serviceobj_servicetype <> 'auto'
                    and status = 'dropped'
                    and spdfraud_flag != 1
                    and serviceobj_city = 'Hyderabad'
                    group by 1, 2
                      rider  o

In [7]:
rides_data.to_csv('captain_weekly_rides_rating.csv',index=False)

In [8]:
#Start time and end time declaration
CC = [
    ['20210628','20210704']
]

In [9]:
# Fetch the pings related to rides for the calculation of APR

final_df = pd.DataFrame()

for i in CC : 

    dpr_query = """
            select captainid,
            CAST(DATE_TRUNC('week',DATE_PARSE(yyyymmdd, '%Y%m%d')) as varchar) as order_week,
            count(distinct(case when eventtype = 'rider_cancelled' then _id end)) as rider_cancelled_pings,
            count(distinct(case when eventtype = 'dropped' then _id end)) as net_rides,
            count(distinct(case when eventtype = 'rider_reject' then _id end)) as rider_rejected_pings,
            count(distinct(case when eventtype = 'rider_busy' then _id end)) as rider_busy_pings,
            count(distinct(case when eventtype = 'accepted' then _id end)) as accepted_pings
            from hive.raw.kafka_order_logs_immutable
            where yyyymmdd >= '{sd}'
            and yyyymmdd <= '{ed}'
            group by 1,2""".format(sd = i[0], ed=i[1])

    print(dpr_query)
    df_dpr = pd.read_sql(dpr_query, presto_conn)
    #df_dpr['yyyymmdd'] = d
    #df_dpr['total_pings'] = df_dpr['accepted_pings'] + df_dpr['rider_busy_pings'] + df_dpr['rider_rejected_pings']
    
    final_df = pd.concat([final_df,df_dpr])
    print(len(final_df))
    final_df.to_csv('Rider_Pings.csv',index=False)
    


            select captainid,
            CAST(DATE_TRUNC('week',DATE_PARSE(yyyymmdd, '%Y%m%d')) as varchar) as order_week,
            count(distinct(case when eventtype = 'rider_cancelled' then _id end)) as rider_cancelled_pings,
            count(distinct(case when eventtype = 'dropped' then _id end)) as net_rides,
            count(distinct(case when eventtype = 'rider_reject' then _id end)) as rider_rejected_pings,
            count(distinct(case when eventtype = 'rider_busy' then _id end)) as rider_busy_pings,
            count(distinct(case when eventtype = 'accepted' then _id end)) as accepted_pings
            from hive.raw.kafka_order_logs_immutable
            where yyyymmdd >= '20210628'
            and yyyymmdd <= '20210704'
            group by 1,2
127393


In [10]:
# Captain LTR calculation

captain_ltr_query = """   SELECT  captainid, day, sum(lifetimenetrides) as LTR
    from 
    datasets_internal.captain_servicedetail_ridebehavior_immutable_v1 
    WHERE 
    day in (date '2021-04-04', date '2021-04-11', date '2021-04-18', date '2021-04-25',
            date '2021-05-02', date '2021-05-09', date '2021-05-16', date '2021-05-23', date '2021-05-30',
            date '2021-06-06', date '2021-06-13', date '2021-06-20')
    group by 1, 2"""

print(captain_ltr_query)
captain_ltr = pd.read_sql(captain_ltr_query, presto_conn)
captain_ltr.head()

   SELECT  captainid, day, sum(lifetimenetrides) as LTR
    from 
    datasets_internal.captain_servicedetail_ridebehavior_immutable_v1 
    WHERE 
    day in (date '2021-07-04')
    group by 1, 2


,captainid,day,LTR
0,5edda8259adf08bfc9f69c27,2021-07-04,24
1,5f437c994980e55497c3d22c,2021-07-04,113
2,5d66a8c7d0286d106d69be34,2021-07-04,981
3,5b504ea10b7ac33b2280806a,2021-07-04,3
4,5d89c090f354c15d38ca5c02,2021-07-04,1370


In [11]:
len(captain_ltr)

1235115

In [12]:
captain_ltr['week'] = pd.to_datetime(captain_ltr['day'])
#captain_ltr['week'] = captain_ltr['week'] + timedelta(1)
captain_ltr.head()

,captainid,day,LTR,week
0,5edda8259adf08bfc9f69c27,2021-07-04,24,2021-07-04
1,5f437c994980e55497c3d22c,2021-07-04,113,2021-07-04
2,5d66a8c7d0286d106d69be34,2021-07-04,981,2021-07-04
3,5b504ea10b7ac33b2280806a,2021-07-04,3,2021-07-04
4,5d89c090f354c15d38ca5c02,2021-07-04,1370,2021-07-04


In [13]:
captain_ltr['week'] = captain_ltr['week'].apply(lambda x : x.strftime('%Y-%V'))
captain_ltr.head()

,captainid,day,LTR,week
0,5edda8259adf08bfc9f69c27,2021-07-04,24,2021-26
1,5f437c994980e55497c3d22c,2021-07-04,113,2021-26
2,5d66a8c7d0286d106d69be34,2021-07-04,981,2021-26
3,5b504ea10b7ac33b2280806a,2021-07-04,3,2021-26
4,5d89c090f354c15d38ca5c02,2021-07-04,1370,2021-26


In [14]:
captain_ltr.to_csv('Captain_LTR_weekly.csv',index=False)

In [75]:
dstart =  pd.to_datetime('2021-01-01')
dend =  pd.to_datetime('2021-03-31')

date_list = [dstart + timedelta(days=x) for x in range((dend-dstart).days + 1)
        if (dstart + timedelta(days=x)).weekday() == 6]

date_list = [x.strftime('%Y-%m-%d') for x in date_list]
date_list

['2021-01-03',
 '2021-01-10',
 '2021-01-17',
 '2021-01-24',
 '2021-01-31',
 '2021-02-07',
 '2021-02-14',
 '2021-02-21',
 '2021-02-28',
 '2021-03-07',
 '2021-03-14',
 '2021-03-21',
 '2021-03-28']

In [76]:

def get_rf_segments(start):
    
    print("fetching rf Segments data for",start)

    orders_query = """SELECT captainid as rider, day, frequency_segment, recency_segment
                      from datasets.captain_link_immutable
                      where day = date('{sd}')
                      AND segment = 'RF'                      
               """.format( sd = start)
    
#     print(orders_query)

    df_orders_data = pd.read_sql(orders_query, presto_conn)
    
    print(" fetching completed for rf segments data", len(df_orders_data))

    return df_orders_data

rf_segments = pd.DataFrame()

for day in date_list : 

    week_rf = get_rf_segments(day)
    rf_segments = pd.concat([rf_segments,week_rf])
    


rf_segments['recency_segment'] = rf_segments['recency_segment'].apply(lambda x : x.title())
rf_segments['frequency_segment'] = rf_segments['frequency_segment'].apply(lambda x : x.title())
rf_segments['Segment'] = rf_segments['recency_segment'] +'-'+ rf_segments['frequency_segment']
rf_segments['week'] = pd.to_datetime(rf_segments['day']).apply(lambda x : (x + timedelta(1)).strftime('%Y-%V'))
#rf_segments['week'] = pd.to_datetime(rf_segments['day']).apply(lambda x : x.strftime('%Y-%V'))
rf_segments

fetching rf Segments data for 2021-01-03
 fetching completed for rf segments data 178253
fetching rf Segments data for 2021-01-10
 fetching completed for rf segments data 178239
fetching rf Segments data for 2021-01-17
 fetching completed for rf segments data 179151
fetching rf Segments data for 2021-01-24
 fetching completed for rf segments data 181670
fetching rf Segments data for 2021-01-31
 fetching completed for rf segments data 183587
fetching rf Segments data for 2021-02-07
 fetching completed for rf segments data 186024
fetching rf Segments data for 2021-02-14
 fetching completed for rf segments data 189475
fetching rf Segments data for 2021-02-21
 fetching completed for rf segments data 191620
fetching rf Segments data for 2021-02-28
 fetching completed for rf segments data 194546
fetching rf Segments data for 2021-03-07
 fetching completed for rf segments data 195506
fetching rf Segments data for 2021-03-14
 fetching completed for rf segments data 195922
fetching rf Segments 

,rider,day,frequency_segment,recency_segment,Segment,week
0,5a0cf5e2727a4a3ef3d10e1d,2021-01-03,Midperforming,Recent,Recent-Midperforming,2021-01
1,5b45932a7ea5462554cc77cf,2021-01-03,Underperforming,Recent,Recent-Underperforming,2021-01
2,5b47501be6982857b8ba0566,2021-01-03,Underperforming,Recent,Recent-Underperforming,2021-01
3,5bac9c0ffee607761aaf936e,2021-01-03,Underperforming,Recent,Recent-Underperforming,2021-01
4,5bea64b4e9a4e0588b3d3d7b,2021-01-03,Highperforming,Recent,Recent-Highperforming,2021-01
...,...,...,...,...,...,...
198214,5c835fc98c352421eae0da52,2021-03-28,Midperforming,Dormant,Dormant-Midperforming,2021-13
198215,5cc828d93d65ca5e2566b5b0,2021-03-28,Midperforming,Dormant,Dormant-Midperforming,2021-13
198216,5cd78ac6377155163cf79832,2021-03-28,Midperforming,Dormant,Dormant-Midperforming,2021-13
198217,5dd7658753a0f569a718be7b,2021-03-28,Underperforming,Inactive,Inactive-Underperforming,2021-13


In [77]:
rf_segments = rf_segments[['rider','week','Segment']]
rf_segments
rf_segments.to_csv('rf_segments.csv',index=False)

In [22]:
rides_data = pd.read_csv('captain_weekly_rides_rating.csv')

rides_data['week'] = pd.to_datetime(rides_data['order_week'])
rides_data['week'] = rides_data['week'].apply(lambda x : x.strftime('%Y-%V'))
rides_data.head()

,rider,order_week,wdays,all_service_rides,rating,week
0,5cbc05d354bc7263ff44d5da,2021-06-28,3,7,4.5,2021-26
1,5cba19a754bc7263ff4289d0,2021-06-28,1,7,5.0,2021-26
2,5d9a1bcfc2153814dfc28cbc,2021-06-28,4,12,5.0,2021-26
3,5fdf10504319a7957eb24ce8,2021-06-28,7,108,4.6,2021-26
4,601b90cf714323cca9c81eb7,2021-06-28,1,2,0.0,2021-26


In [21]:
rider_cancelled = pd.read_csv('Rider_Pings.csv')
rider_cancelled.head()

,captainid,order_week,rider_cancelled_pings,net_rides,rider_rejected_pings,rider_busy_pings,accepted_pings
0,5fe9dffc900d7658c06551fe,2021-06-28 00:00:00.000,2,9,6,3,17
1,5f74ba3c2508b6f8478f2988,2021-06-28 00:00:00.000,5,32,0,2,69
2,5d3dad0cbcfa850af7c46315,2021-06-28 00:00:00.000,2,5,1,3,8
3,60632c0601f9655b6c973d6d,2021-06-28 00:00:00.000,0,17,0,4,21
4,60097829ae1f6b3b219f3c23,2021-06-28 00:00:00.000,1,0,10,2,1


In [20]:
captain_ltr = pd.read_csv('Captain_LTR_weekly.csv')

In [13]:
### Captain Weekly LTR

captain_ltr['ltr_bucket'] = captain_ltr['LTR'].apply(lambda x : '01-05' if x <= 5 else 
                                                   ('6-50' if (x>=6) and (x<=50) 
                                                    else('50-100' if (x>50) and (x<=100) else
                                                        ('100-250' if (x>100) and (x<=250) else
                                                        ('250-550' if (x>250) and (x<=550) else 
                                                        (('550-800' if (x>550) and (x<=800) else 
                                                          (('800-1200' if (x>800) and (x<=1200) else 
                                                          '1200+')))))))))
print(captain_ltr.head())
     
captain_ltr = captain_ltr[['captainid','week','LTR','ltr_bucket']]
captain_ltr

                  captainid         day  LTR     week ltr_bucket
0  5db28cd348c3c9114d3a1198  2021-05-23  617  2021-20    550-800
1  5e3a699586bdf51b1290e878  2021-05-23  280  2021-20    250-550
2  5c59aaa703a62d32bb1c943a  2021-05-23    1  2021-20      01-05
3  5c12385aa24a6608213aeb35  2021-05-23    8  2021-20       6-50
4  573f29169b0ffc2836777c15  2021-05-23    6  2021-20       6-50


,captainid,week,LTR,ltr_bucket
0,5db28cd348c3c9114d3a1198,2021-20,617,550-800
1,5e3a699586bdf51b1290e878,2021-20,280,250-550
2,5c59aaa703a62d32bb1c943a,2021-20,1,01-05
3,5c12385aa24a6608213aeb35,2021-20,8,6-50
4,573f29169b0ffc2836777c15,2021-20,6,6-50
...,...,...,...,...
14447416,5a1e725fc7de0e141dd82b67,2021-20,3,01-05
14447417,5e589d529dc80573d8a212ce,2021-20,48,6-50
14447418,5f6ecac97d99a7a62e566bb9,2021-20,196,100-250
14447419,5f3f97377e96723eb9005e7a,2021-20,165,100-250


In [23]:
### DPR Data

#dpr = pd.read_csv('DPR.csv')
#dpr['yyyymmdd'] = dpr['yyyymmdd'].apply(str)
#rider_cancelled['yyyymmdd'] = rider_cancelled['yyyymmdd'].apply(str)


#dpr = dpr.merge(rider_cancelled, how='left', on=['captainid','yyyymmdd'])

#dpr['week'] = dpr['yyyymmdd'].apply(lambda x : datetime.strptime(x, '%Y%m%d').strftime('%Y-%V'))

#dpr_week = dpr.groupby(['captainid','week'],as_index=False).agg({'net_rides':'sum',
#                                                                  'accepted_pings':'sum',
#                                                                  'rider_busy_pings':'sum',
#                                                                  'rider_reject_pings':'sum',
#                                                                  'rider_cancelled_pings':'sum'})

#rider_cancelled['week'] = rider_cancelled['yyyymmdd'].apply(lambda x : datetime.strptime(x, '%Y%m%d').strftime('%Y-%V'))


rider_cancelled['week'] = pd.to_datetime(rider_cancelled['order_week'])
rider_cancelled['week'] = rider_cancelled['week'].apply(lambda x : x.strftime('%Y-%V'))

rider_cancelled_week = rider_cancelled.groupby(['captainid','week'],as_index=False).agg({'net_rides':'sum',
                                                                  'accepted_pings':'sum',
                                                                  'rider_busy_pings':'sum',
                                                                  'rider_rejected_pings':'sum',
                                                                  'rider_cancelled_pings':'sum'})


rider_cancelled_week.head()

,captainid,week,net_rides,accepted_pings,rider_busy_pings,rider_rejected_pings,rider_cancelled_pings
0,5737c6bfddbec2203f73320c,2021-26,0,25,3,1,11
1,5737c716ddbec2203f7334fd,2021-26,0,1,1,4,0
2,5737c8d7ddbec220a7fa1738,2021-26,2,9,0,0,0
3,5737ca7fddbec220de03eb96,2021-26,41,41,41,93,0
4,5737ca92ddbec220de03ec37,2021-26,0,0,5,7,0


In [15]:
### Cancel Rate Data - taken from dashboard DPR Tracker: Captain List

cancel_rate_1 = pd.read_csv('Cancel_Mar29.csv')
cancel_rate_1['week'] = '2021-13'

cancel_rate_2 = pd.read_csv('Cancel_Apr5.csv')
cancel_rate_2['week'] = '2021-14'

cancel_rate_3 = pd.read_csv('Cancel_Apr12.csv')
cancel_rate_3['week'] = '2021-15'

cancel_rate_4 = pd.read_csv('Cancel_Apr19.csv')
cancel_rate_4['week'] = '2020-16'

cancel_rate_5 = pd.read_csv('Cancel_Apr26.csv')
cancel_rate_5['week'] = '2020-17'

cancel_rate_6 = pd.read_csv('Cancel_May3.csv')
cancel_rate_6['week'] = '2020-18'

cancel_rate_7 = pd.read_csv('Cancel_May10.csv')
cancel_rate_7['week'] = '2020-19'

cancel_rate_8 = pd.read_csv('Cancel_May17.csv')
cancel_rate_8['week'] = '2020-20'

cancel_rate_9 = pd.read_csv('Cancel_May24.csv')
cancel_rate_9['week'] = '2020-21'

cancel_rate_10 = pd.read_csv('Cancel_May31.csv')
cancel_rate_10['week'] = '2021-22'

cancel_rate_11 = pd.read_csv('Cancel_Jun7.csv')
cancel_rate_11['week'] = '2021-23'

cancel_rate_12 = pd.read_csv('Cancel_Jun14.csv')
cancel_rate_12['week'] = '2021-24'

#cancel_rate_13 = pd.read_csv('Cancel_Jan25.csv')
#cancel_rate_13['week'] = '2021-04'

#cancel_rate_14 = pd.read_csv('Cancel_Feb1.csv')
#cancel_rate_14['week'] = '2021-05'

#cancel_rate_15 = pd.read_csv('Cancel_Feb8.csv')
#cancel_rate_15['week'] = '2021-06'

#cancel_rate_16 = pd.read_csv('Cancel_Feb15.csv')
#cancel_rate_16['week'] = '2021-07'

#cancel_rate_17 = pd.read_csv('Cancel_Feb22.csv')
#cancel_rate_17['week'] = '2021-08'

#cancel_rate_18 = pd.read_csv('Cancel_Mar1.csv')
#cancel_rate_18['week'] = '2021-09'

#cancel_rate_19 = pd.read_csv('Cancel_Mar8.csv')
#cancel_rate_19['week'] = '2021-10'

#cancel_rate_20 = pd.read_csv('Cancel_Mar15.csv')
#cancel_rate_20['week'] = '2021-11'

#cancel_rate_21 = pd.read_csv('Cancel_Mar22.csv')
#cancel_rate_21['week'] = '2021-12'

cancel_rate = pd.concat([cancel_rate_1,cancel_rate_2,cancel_rate_3,cancel_rate_4,cancel_rate_5,cancel_rate_6,cancel_rate_7,cancel_rate_8,cancel_rate_9,
                        cancel_rate_10,cancel_rate_11,cancel_rate_12])

cancel_rate = cancel_rate.groupby(['captain_id','week'],as_index=False).agg({'Net':'sum',
                                                                              'Captain-induced CC Pings':'sum'
                                                                             })
cancel_rate = cancel_rate[['captain_id','Net','Captain-induced CC Pings','week']]
cancel_rate

,captain_id,Net,Captain-induced CC Pings,week
0,5737c6aeddbec2203f733176,0,0,2020-17
1,5737c6bfddbec2203f73320c,9,2,2020-17
2,5737c6bfddbec2203f73320c,4,0,2020-18
3,5737c6bfddbec2203f73320c,5,0,2020-19
4,5737c6bfddbec2203f73320c,0,1,2020-20
...,...,...,...,...
1258923,60cef11f0546893239dbf5c3,0,0,2021-24
1258924,60cef9b8f5cf11799192fba7,0,0,2021-24
1258925,60cf16d4f5cf112c93930d95,0,0,2021-24
1258926,60cf207ef5cf114050931348,0,0,2021-24


In [26]:
final_data = rides_data.merge(rider_cancelled_week, how='left', left_on=['rider'], right_on=['captainid'])
final_data = final_data.merge(cancel_rate, how='left', left_on=['rider'], right_on=['captain_id'])
final_data = final_data.merge(captain_ltr, how='left', left_on=['rider'], right_on=['captainid'])

final_data['cancel'] = (final_data['Captain-induced CC Pings']+final_data['rider_cancelled_pings'])/final_data['accepted_pings'] 
final_data['cancel'] = final_data['cancel'].fillna(0)

final_data['total_pings'] = final_data['accepted_pings']+final_data['rider_busy_pings']+final_data['rider_rejected_pings']
final_data['APR'] = final_data['accepted_pings']/final_data['total_pings']

final_data

,rider,order_week,wdays,all_service_rides,rating,week_x,captainid_x,week_y,net_rides,accepted_pings,...,captain_id,Net,Captain-induced CC Pings,captainid_y,day,LTR,week,cancel,total_pings,APR
0,5cbc05d354bc7263ff44d5da,2021-06-28,3,7,4.5,2021-26,5cbc05d354bc7263ff44d5da,2021-26,7,14,...,5cbc05d354bc7263ff44d5da,7,0,5cbc05d354bc7263ff44d5da,2021-07-04,61,2021-26,0.285714,74,0.189189
1,5cba19a754bc7263ff4289d0,2021-06-28,1,7,5.0,2021-26,5cba19a754bc7263ff4289d0,2021-26,7,12,...,5cba19a754bc7263ff4289d0,7,0,5cba19a754bc7263ff4289d0,2021-07-04,297,2021-26,0.166667,16,0.750000
2,5d9a1bcfc2153814dfc28cbc,2021-06-28,4,12,5.0,2021-26,5d9a1bcfc2153814dfc28cbc,2021-26,12,27,...,5d9a1bcfc2153814dfc28cbc,12,0,5d9a1bcfc2153814dfc28cbc,2021-07-04,58,2021-26,0.185185,79,0.341772
3,5fdf10504319a7957eb24ce8,2021-06-28,7,108,4.6,2021-26,5fdf10504319a7957eb24ce8,2021-26,108,145,...,5fdf10504319a7957eb24ce8,108,2,5fdf10504319a7957eb24ce8,2021-07-04,174,2021-26,0.020690,168,0.863095
4,601b90cf714323cca9c81eb7,2021-06-28,1,2,0.0,2021-26,601b90cf714323cca9c81eb7,2021-26,2,7,...,601b90cf714323cca9c81eb7,2,0,601b90cf714323cca9c81eb7,2021-07-04,36,2021-26,0.571429,16,0.437500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12389,60c2fd5b48084d1a0a08378e,2021-06-28,1,1,0.0,2021-26,60c2fd5b48084d1a0a08378e,2021-26,19,26,...,60c2fd5b48084d1a0a08378e,19,2,60c2fd5b48084d1a0a08378e,2021-07-04,56,2021-26,0.230769,137,0.189781
12390,6045c4b6dc56982d09bff1bd,2021-06-28,1,1,5.0,2021-26,6045c4b6dc56982d09bff1bd,2021-26,1,3,...,6045c4b6dc56982d09bff1bd,1,0,6045c4b6dc56982d09bff1bd,2021-07-04,251,2021-26,0.000000,3,1.000000
12391,5d348c6584e7394e94742700,2021-06-28,1,1,5.0,2021-26,5d348c6584e7394e94742700,2021-26,1,1,...,5d348c6584e7394e94742700,1,0,5d348c6584e7394e94742700,2021-07-04,841,2021-26,0.000000,6,0.166667
12392,60022f83857d1b99442b7b5e,2021-06-28,1,1,0.0,2021-26,60022f83857d1b99442b7b5e,2021-26,1,1,...,60022f83857d1b99442b7b5e,1,0,60022f83857d1b99442b7b5e,2021-07-04,1,2021-26,0.000000,1,1.000000


In [27]:
final_data.to_csv('Hyderabad_Captains.csv')

In [25]:
final_data = rides_data.merge(rider_cancelled_week, how='left', left_on=['rider','week'], right_on=['captainid','week'])
final_data = final_data.merge(cancel_rate, how='left', left_on=['rider','week'], right_on=['captain_id','week'])
final_data = final_data.merge(captain_ltr, how='left', left_on=['rider','week'], right_on=['captainid','week'])

final_data['cancel'] = (final_data['Captain-induced CC Pings']+final_data['rider_cancelled_pings'])/final_data['accepted_pings'] 
final_data['cancel'] = final_data['cancel'].fillna(0)

final_data['total_pings'] = final_data['accepted_pings']+final_data['rider_busy_pings']+final_data['rider_rejected_pings']
final_data['APR'] = final_data['accepted_pings']/final_data['total_pings']

final_data

KeyError: 'week'

In [17]:
final_data.isna().sum()

rider                            0
order_week                       0
wdays                            0
all_service_rides                0
rating                           0
week                             0
captainid_x                      0
net_rides                        0
accepted_pings                   0
rider_busy_pings                 0
rider_rejected_pings             0
rider_cancelled_pings            0
captain_id                  253723
Net                         253723
Captain-induced CC Pings    253723
captainid_y                      0
LTR                              0
ltr_bucket                       0
cancel                           0
total_pings                      0
APR                              0
dtype: int64

In [18]:
final_data['ltr_bucket'] = final_data['ltr_bucket'].fillna('01-05')
final_data.isna().sum()

rider                            0
order_week                       0
wdays                            0
all_service_rides                0
rating                           0
week                             0
captainid_x                      0
net_rides                        0
accepted_pings                   0
rider_busy_pings                 0
rider_rejected_pings             0
rider_cancelled_pings            0
captain_id                  253723
Net                         253723
Captain-induced CC Pings    253723
captainid_y                      0
LTR                              0
ltr_bucket                       0
cancel                           0
total_pings                      0
APR                              0
dtype: int64

In [19]:
final_data.to_csv('Captain_Levels_Weekly_Data.csv',index=False)

In [21]:
final_data.week.unique()

array(['2021-17', '2021-19', '2021-22', '2021-24', '2021-20', '2021-23',
       '2021-18', '2021-16', '2021-15', '2021-14', '2021-21', '2021-13'],
      dtype=object)

In [20]:
final_data = pd.read_csv('Captain_Levels_Weekly_Data.csv')

In [21]:
input_data = pd.read_csv('delhi_config.csv')
input_data

,ltr_bucket,rating,all_service_rides,APR,cancel,active_days,priority,Captain Level
0,1-5,0.0,0,0.00,1.00,0,8,Grey
1,6-50,4.0,3,0.50,0.20,5,7,Blue
2,50-100,4.1,6,0.55,0.17,10,6,Bronze
3,100-250,4.2,15,0.60,0.15,15,5,Silver
4,250-550,4.3,25,0.65,0.14,18,4,Gold
5,550-800,4.4,35,0.70,0.12,20,3,Diamond
6,800-1200,4.5,45,0.75,0.10,22,2,Platinum
7,1200+,4.6,55,0.80,0.10,25,1,Elite


In [22]:
### Including all parameters :


data_sql_query = ps.sqldf("select lt.*, ft.priority from final_data lt left join input_data ft on lt.all_service_rides >= ft.all_service_rides and lt.rating >= ft.rating and lt.APR >= ft.APR and lt.cancel <= ft.cancel")

max_p_check =  data_sql_query.merge(input_data[['ltr_bucket','priority']].rename(columns={'priority':'max_priority'}), how='left', on=['ltr_bucket'])

max_p_check['priority_check'] = max_p_check['priority'] >= max_p_check['max_priority']

max_p_check = max_p_check[max_p_check['priority_check']==True]

priority_table = max_p_check.groupby(['rider','ltr_bucket', 'all_service_rides', 'rating', 'max_priority', 'APR','cancel', 'week']).agg({'priority':'min'})
priority_table = priority_table.reset_index()
priority_table['priority'] = priority_table['priority'].fillna(0)
priority_table


,rider,ltr_bucket,all_service_rides,rating,max_priority,APR,cancel,week,priority
0,5737c6bfddbec2203f73320c,1200+,1,0.0,1.0,0.888889,0.562500,2021-13,8.0
1,5737c6bfddbec2203f73320c,1200+,1,5.0,1.0,0.800000,0.375000,2021-15,8.0
2,5737c6bfddbec2203f73320c,1200+,3,5.0,1.0,0.785714,0.000000,2021-21,7.0
3,5737c6bfddbec2203f73320c,1200+,4,0.0,1.0,0.809524,0.000000,2021-18,8.0
4,5737c6bfddbec2203f73320c,1200+,5,5.0,1.0,0.892857,0.000000,2021-19,7.0
...,...,...,...,...,...,...,...,...,...
606711,60cd9322f5cf11d63e926033,6-50,8,4.5,7.0,0.888889,0.000000,2021-24,7.0
606712,60cdc04ef5cf111f469280d0,6-50,10,4.0,7.0,0.705882,0.000000,2021-24,7.0
606713,60cdcc0ff5cf11e1c3928883,6-50,13,5.0,7.0,1.000000,0.000000,2021-24,7.0
606714,60ce02f8054689c7b1dba906,6-50,6,5.0,7.0,1.000000,0.111111,2021-24,7.0


In [23]:
#priority_table = priority_table[priority_table['week']=='2021-12']
priority_table.to_csv('priority_table.csv',index=False)

In [24]:
#Start time and end time declaration
CC = [['2021-02-01','2021-02-07']]
xity='Delhi'

In [277]:
#Fetch the weekly incentive data for Captains

final_df = pd.DataFrame()

for i in CC : 

    incentive_query = """select incentive.riderid, CAST(DATE_TRUNC('week',CAST(o.orderdate AS DATE)) as varchar) as order_week, sum(incentive.ipr) as weekly_incentive, count(distinct incentive.order_id) as weekly_incentive_order_count
    from
    (select yy.riderid, yy._id, yy.yyyymmdd, yy.payment_date, yy.city, yy.amount, yy.order_count, yy.service_name, yy.ipr, replace(y.orderid,'"','') as order_id
        from
            (select xx.*, x.order_count, x.service_name, cast(xx.amount as double)/xx.total_order as ipr, regexp_extract_all(json_format(x.order_id), '"[a-z0-9]{24}"') as order_id
                from
                    (with ti as
                        (select riderId as riderid, _id, yyyymmdd, paymentDate as payment_date, city, amount, incentiveDetails_orderCount, incentiveDetails_service, incentiveDetails_orderIds, Dense_Rank() over(partition by _id order by updated_epoch desc) as R
                            from hive.raw.kafka_captain_transactions_immutable where status = 'success' and transactionCategory = 'specialIncentive' and upper(city) = 'DELHI' and yyyymmdd between '20210614' and '20210622'),
                    ri as 
                        (select riderId, _id, yyyymmdd, tincentiveIdl, incentiveStage, Dense_Rank() over(partition by _id order by updated_epoch desc) as R
                            from raw.mongodb_rapidopayroll_riderspaymentnew_immutable where incentiveType = 'Weekly Fixed' and status = 'success' and upper(city) = 'DELHI' and yyyymmdd between '20210614' and '20210622')
                    
                    select ti.riderId as riderid,
                            ti._id, 
                            ti.yyyymmdd, 
                            ti.payment_date, 
                            ti.city, 
                            ti.amount,
                            reduce(cast(ti.incentiveDetails_orderCount as array(double)),0, (s, x) -> s + x, s -> s) as total_order,
                            ti.incentiveDetails_orderCount,
                            ti.incentiveDetails_service, 
                            ti.incentiveDetails_orderIds,
                            ri.tincentiveIdl as incentive_id,
                            ri.incentiveStage as incentive_stage
                        from ti
                            join ri
                                on ti.riderId = ri.riderId and ti._id = ri._id and ti.yyyymmdd = ri.yyyymmdd
                        where ti.R = 1
                            and ri.R = 1
                    ) xx cross join unnest(incentiveDetails_orderCount, incentiveDetails_service, incentiveDetails_orderIds) as x(order_count, service_name, order_id)
            ) yy cross join unnest(yy.order_id) as y(orderid)
    ) incentive
    left join legacy.orders o
        on incentive.order_id = o._id
    where o.orderdate between '2021-06-14' and '2021-06-20'
    group by 1,2"""#.format(city = xity, sd = i[0] , ed = i[1])

    print(incentive_query)
    df_incentive = pd.read_sql(incentive_query, presto_conn)
    
    final_df = pd.concat([final_df,df_incentive])
    print(len(final_df))

select incentive.riderid, CAST(DATE_TRUNC('week',CAST(o.orderdate AS DATE)) as varchar) as order_week, sum(incentive.ipr) as weekly_incentive, count(distinct incentive.order_id) as weekly_incentive_order_count
    from
    (select yy.riderid, yy._id, yy.yyyymmdd, yy.payment_date, yy.city, yy.amount, yy.order_count, yy.service_name, yy.ipr, replace(y.orderid,'"','') as order_id
        from
            (select xx.*, x.order_count, x.service_name, cast(xx.amount as double)/xx.total_order as ipr, regexp_extract_all(json_format(x.order_id), '"[a-z0-9]{24}"') as order_id
                from
                    (with ti as
                        (select riderId as riderid, _id, yyyymmdd, paymentDate as payment_date, city, amount, incentiveDetails_orderCount, incentiveDetails_service, incentiveDetails_orderIds, Dense_Rank() over(partition by _id order by updated_epoch desc) as R
                            from hive.raw.kafka_captain_transactions_immutable where status = 'success' and trans

In [20]:
#Fetch the weekly incentive data for Captains

final_df = pd.DataFrame()

for i in CC : 

    incentive_query = """select riderid, 
        yyyymmdd, 
        sum(incentive) as weekly_incentive, 
        sum(order_count) as weekly_incentive_order_count
    FROM
        (SELECT riderid, 
                       yyyymmdd, 
                       CAST(amount as DOUBLE) as incentive,
                       CAST(orderCount as DOUBLE) as order_count,
                       Dense_Rank() over(partition by _id order by updated_epoch desc) as R
                FROM raw.mongodb_rapidopayroll_riderspaymentnew_immutable
                WHERE yyyymmdd >= '20210201'
                    AND yyyymmdd <= '20210207'
                    AND city = 'Hyderabad'
                    AND transactiontype = 'specialIncentive'
                    AND incentivetype = 'Weekly Fixed'
                    AND status = 'success'
        ) x
    where R = 1
    group by 1,2"""#.format(city = xity, sd = i[0] , ed = i[1])

    print(incentive_query)
    df_incentive = pd.read_sql(incentive_query, presto_conn)
    
    final_df = pd.concat([final_df,df_incentive])
    print(len(final_df))

select riderid, 
        yyyymmdd, 
        sum(incentive) as weekly_incentive, 
        sum(order_count) as weekly_incentive_order_count
    FROM
        (SELECT riderid, 
                       yyyymmdd, 
                       CAST(amount as DOUBLE) as incentive,
                       CAST(orderCount as DOUBLE) as order_count,
                       Dense_Rank() over(partition by _id order by updated_epoch desc) as R
                FROM raw.mongodb_rapidopayroll_riderspaymentnew_immutable
                WHERE yyyymmdd >= '20210201'
                    AND yyyymmdd <= '20210207'
                    AND city = 'Hyderabad'
                    AND transactiontype = 'specialIncentive'
                    AND incentivetype = 'Weekly Fixed'
                    AND status = 'success'
        ) x
    where R = 1
    group by 1,2
768


In [21]:
df_weekly_incentive = final_df.copy()

df_weekly_incentive['yyyymmdd'] = df_weekly_incentive['yyyymmdd'].apply(str)

df_weekly_incentive['week'] = df_weekly_incentive['yyyymmdd'].apply(lambda x : datetime.strptime(x, '%Y%m%d').strftime('%Y-%V'))

df_weekly_incentive = df_weekly_incentive[['riderid','week','weekly_incentive','weekly_incentive_order_count']]

df_weekly_incentive = df_weekly_incentive.groupby(['riderid','week']).agg({'weekly_incentive':'sum','weekly_incentive_order_count':'sum'}).reset_index()
df_weekly_incentive['weekly_ipr'] = df_weekly_incentive['weekly_incentive']/df_weekly_incentive['weekly_incentive_order_count']

df_weekly_incentive.to_csv('weekly_incentive.csv',index=False)
df_weekly_incentive.head(5)

,riderid,week,weekly_incentive,weekly_incentive_order_count,weekly_ipr
0,57b3fa789e81677e7efa0e74,2021-05,150.0,20.0,7.50
1,5a158776bb110a2f4e403414,2021-05,120.0,25.0,4.80
2,5a23c5b74b6325587cc54645,2021-05,120.0,25.0,4.80
3,5a7d721ebd10ab1be6eded05,2021-05,150.0,20.0,7.50
4,5a944ef4c5a70933a82835b8,2021-05,350.0,40.0,8.75


In [74]:
final_df.head(5)

,riderid,order_week,weekly_incentive,weekly_incentive_order_count
0,5c708745de141933060f97ed,2021-06-07,628.0,60
1,5d8bb577a1f0a71c6047ae1f,2021-06-07,400.0,35
2,5d9c5f2561bf5a14d0957ea7,2021-06-07,250.0,20
3,5f29f69c4b3a152112c5ce5e,2021-06-07,250.0,20
4,5dc559df907d93538e3b4cdf,2021-06-07,250.0,20


In [293]:
df_weekly_incentive = final_df.copy()

df_weekly_incentive['week'] = pd.to_datetime(df_weekly_incentive['order_week']).apply(lambda x : x.strftime('%Y-%V'))

df_weekly_incentive = df_weekly_incentive[['riderid','week','weekly_incentive','weekly_incentive_order_count']]

df_weekly_incentive = df_weekly_incentive.groupby(['riderid','week']).agg({'weekly_incentive':'sum','weekly_incentive_order_count':'sum'}).reset_index()
df_weekly_incentive['weekly_ipr'] = df_weekly_incentive['weekly_incentive']/df_weekly_incentive['weekly_incentive_order_count']

df_weekly_incentive.to_csv('weekly_incentive_Delhi.csv',index=False)
df_weekly_incentive.head(5)

,riderid,week,weekly_incentive,weekly_incentive_order_count,weekly_ipr
0,57a7c05fc59d16a606a3b288,2021-24,275.0,29,9.482759
1,587acd5d34af95807c6b0ec1,2021-24,380.0,38,10.000000
2,5951da6d39f6c787733a22d2,2021-24,292.0,30,9.733333
3,59de1566977bf007072709c4,2021-24,192.0,24,8.000000
4,5a0d5d28ce3dad6b606187f9,2021-24,280.0,35,8.000000


In [79]:
#Fetch the Daily incentive data for Captains

final_df = pd.DataFrame()

for i in CC : 

    incentive_query = """select incentive.riderid, CAST(DATE_TRUNC('week',CAST(o.orderdate AS DATE)) as varchar) as order_week, sum(incentive.ipr) as daily_incentive, count(distinct incentive.order_id) as daily_incentive_order_count
    from
    (select yy.riderid, yy._id, yy.yyyymmdd, yy.payment_date, yy.city, yy.amount, yy.order_count, yy.service_name, yy.ipr, replace(y.orderid,'"','') as order_id
        from
            (select xx.*, x.order_count, x.service_name, cast(xx.amount as double)/xx.total_order as ipr, regexp_extract_all(json_format(x.order_id), '"[a-z0-9]{24}"') as order_id
                from
                    (with ti as
                        (select riderId as riderid, _id, yyyymmdd, paymentDate as payment_date, city, amount, incentiveDetails_orderCount, incentiveDetails_service, incentiveDetails_orderIds, Dense_Rank() over(partition by _id order by updated_epoch desc) as R
                            from hive.raw.kafka_captain_transactions_immutable where status = 'success' and transactionCategory = 'specialIncentive' and upper(city) = 'DELHI' and yyyymmdd between '20210614' and '20210622'),
                    ri as 
                        (select riderId, _id, yyyymmdd, tincentiveIdl, incentiveStage, Dense_Rank() over(partition by _id order by updated_epoch desc) as R
                            from raw.mongodb_rapidopayroll_riderspaymentnew_immutable where incentiveType = 'Daily' and status = 'success' and upper(city) = 'DELHI' and yyyymmdd between '20210614' and '20210622')
                    
                    select ti.riderId as riderid,
                            ti._id, 
                            ti.yyyymmdd, 
                            ti.payment_date, 
                            ti.city, 
                            ti.amount,
                            reduce(cast(ti.incentiveDetails_orderCount as array(double)),0, (s, x) -> s + x, s -> s) as total_order,
                            ti.incentiveDetails_orderCount,
                            ti.incentiveDetails_service, 
                            ti.incentiveDetails_orderIds,
                            ri.tincentiveIdl as incentive_id,
                            ri.incentiveStage as incentive_stage
                        from ti
                            join ri
                                on ti.riderId = ri.riderId and ti._id = ri._id and ti.yyyymmdd = ri.yyyymmdd
                        where ti.R = 1
                            and ri.R = 1
                    ) xx cross join unnest(incentiveDetails_orderCount, incentiveDetails_service, incentiveDetails_orderIds) as x(order_count, service_name, order_id)
            ) yy cross join unnest(yy.order_id) as y(orderid)
    ) incentive
    left join legacy.orders o
        on incentive.order_id = o._id
    where o.orderdate between '2021-06-14' and '2021-06-20'
    group by 1,2"""#.format(city = xity, sd = i[0] , ed = i[1])

    print(incentive_query)
    df_incentive = pd.read_sql(incentive_query, presto_conn)
    
    final_df = pd.concat([final_df,df_incentive])
    print(len(final_df))

select incentive.riderid, CAST(DATE_TRUNC('week',CAST(o.orderdate AS DATE)) as varchar) as order_week, sum(incentive.ipr) as daily_incentive, count(distinct incentive.order_id) as daily_incentive_order_count
    from
    (select yy.riderid, yy._id, yy.yyyymmdd, yy.payment_date, yy.city, yy.amount, yy.order_count, yy.service_name, yy.ipr, replace(y.orderid,'"','') as order_id
        from
            (select xx.*, x.order_count, x.service_name, cast(xx.amount as double)/xx.total_order as ipr, regexp_extract_all(json_format(x.order_id), '"[a-z0-9]{24}"') as order_id
                from
                    (with ti as
                        (select riderId as riderid, _id, yyyymmdd, paymentDate as payment_date, city, amount, incentiveDetails_orderCount, incentiveDetails_service, incentiveDetails_orderIds, Dense_Rank() over(partition by _id order by updated_epoch desc) as R
                            from hive.raw.kafka_captain_transactions_immutable where status = 'success' and transac

In [80]:
df_daily_incentive = final_df.copy()

df_daily_incentive['week'] = pd.to_datetime(df_daily_incentive['order_week']).apply(lambda x : x.strftime('%Y-%V'))
df_daily_incentive['daily_ipr'] = df_daily_incentive['daily_incentive']/df_daily_incentive['daily_incentive_order_count']

df_daily_incentive = df_daily_incentive[['riderid','week','daily_incentive','daily_incentive_order_count','daily_ipr']]
df_daily_incentive.to_csv('daily_incentive_Delhi.csv',index=False)
df_daily_incentive.head(5)

,riderid,week,daily_incentive,daily_incentive_order_count,daily_ipr
0,5f1b0c18bef53d16c426a804,2021-24,72.0,10,7.200000
1,5de4ab3539cbc02e9094fc3b,2021-24,212.0,27,7.851852
2,5d56a810ad6dea53c8e2552f,2021-24,162.0,22,7.363636
3,6066d90cc4a738536cfd7109,2021-24,75.0,9,8.333333
4,60462b092b32d27f18d09396,2021-24,25.0,3,8.333333


In [27]:
#Fetch the Daily incentive data for Captains

final_df = pd.DataFrame()

for i in CC : 

    incentive_query = """select riderid, 
        yyyymmdd, 
        sum(incentive) as daily_incentive, 
        sum(order_count) as daily_incentive_order_count
    FROM
        (SELECT riderid, 
                       yyyymmdd, 
                       CAST(amount as DOUBLE) as incentive,
                       CAST(orderCount as DOUBLE) as order_count,
                       Dense_Rank() over(partition by _id order by updated_epoch desc) as R
                FROM raw.mongodb_rapidopayroll_riderspaymentnew_immutable
                WHERE yyyymmdd >= '20210201'
                    AND yyyymmdd <= '20210207'
                    AND city = 'Hyderabad'
                    AND transactiontype = 'specialIncentive'
                    AND incentivetype = 'Daily'
                    AND status = 'success'
        ) x
    where R = 1
    group by 1,2"""#.format(city = xity, sd = i[0] , ed = i[1])

    print(incentive_query)
    df_incentive = pd.read_sql(incentive_query, presto_conn)
    
    final_df = pd.concat([final_df,df_incentive])
    print(len(final_df))

select riderid, 
        yyyymmdd, 
        sum(incentive) as daily_incentive, 
        sum(order_count) as daily_incentive_order_count
    FROM
        (SELECT riderid, 
                       yyyymmdd, 
                       CAST(amount as DOUBLE) as incentive,
                       CAST(orderCount as DOUBLE) as order_count,
                       Dense_Rank() over(partition by _id order by updated_epoch desc) as R
                FROM raw.mongodb_rapidopayroll_riderspaymentnew_immutable
                WHERE yyyymmdd >= '20210201'
                    AND yyyymmdd <= '20210207'
                    AND city = 'Hyderabad'
                    AND transactiontype = 'specialIncentive'
                    AND incentivetype = 'Daily'
                    AND status = 'success'
        ) x
    where R = 1
    group by 1,2
24632


In [28]:
df_daily_incentive = final_df.copy()

df_daily_incentive['yyyymmdd'] = df_daily_incentive['yyyymmdd'].apply(str)

df_daily_incentive['week'] = df_daily_incentive['yyyymmdd'].apply(lambda x : datetime.strptime(x, '%Y%m%d').strftime('%Y-%V'))

df_daily_incentive = df_daily_incentive[['riderid','week','daily_incentive','daily_incentive_order_count']]

df_daily_incentive = df_daily_incentive.groupby(['riderid','week']).agg({'daily_incentive':'sum','daily_incentive_order_count':'sum'}).reset_index()
df_daily_incentive['daily_ipr'] = df_daily_incentive['daily_incentive']/df_daily_incentive['daily_incentive_order_count']

df_daily_incentive.to_csv('daily_incentive.csv',index=False)
df_daily_incentive.head(5)

,riderid,week,daily_incentive,daily_incentive_order_count,daily_ipr
0,573f29009b0ffc283677244c,2021-05,5.0,1.0,5.000000
1,5784deb13d9b089f0531484a,2021-05,31.0,5.0,6.200000
2,57b3fa789e81677e7efa0e74,2021-05,72.0,6.0,12.000000
3,5803798dbdadd2b9300c8fa3,2021-05,125.0,17.0,7.352941
4,582755a20d21b9eb4bb441f1,2021-05,5.0,1.0,5.000000


In [276]:
# Find the incentive eligible and incentive achieved

def get_incentive_eligible():
    
    #print("fetching eligible incentive for",start)

    incentive_eligible = """with 
    incentive as
                (select incentive_id, 
                        subincentive_id,
                        max("order") as incentive_eligible_orders,
                        sum(amount) as amount
                    from
                    (select z.R, z.incentive_id, z.uuid as subincentive_id, zz.amount, zz.index, zz."order", zz.distance
                        from
                        (
                        select  xx.R, xx.incentive_id, xx.incentive_name, xx.incentive_type, xx.city, xx.start_date, xx.end_date, xx.selected_condition, xx.selected_variable, xx.key, element_at(yy.timeslot,1).fromTime as from_time, element_at(yy.timeslot,1).toTime as to_time, yy.uuid, 
                                cast(yy.rules as array(row("order" integer, amount integer, distance integer, index integer))) as rules
                            from
                            (
                            select x.R, x.incentive_id, x.incentive_name, x.incentive_type, x.city, x.start_date, x.end_date, x.selected_condition, x.selected_variable, y.key, cast(y.value as array(row(rules array(row("order" integer, amount integer, distance integer, index integer)), timeSlot array(row(fromTime varchar, toTime varchar)), uuid varchar))) as value
                                from
                                (
                                select  distinct 
                                        JSON_EXTRACT_SCALAR(JSON_PARSE(data),'$._id["$oid"]') as incentive_id,
                                        JSON_EXTRACT_SCALAR(JSON_PARSE(data),'$.incentiveName') as incentive_name,
                                        JSON_EXTRACT_SCALAR(JSON_PARSE(data),'$.incentiveType') as incentive_type,
                                        JSON_EXTRACT(JSON_PARSE(data),'$.cities') as city,
                                        startDate as start_date,
                                        endDate as end_date,
                                        JSON_EXTRACT(JSON_PARSE(data),'$.serviceNames') as service_name,
                                        JSON_EXTRACT_SCALAR(JSON_PARSE(data),'$.ruleName') as rule_name,
                                        JSON_EXTRACT_SCALAR(JSON_PARSE(data),'$.ruleId') as rule_id,
                                        JSON_EXTRACT_SCALAR(JSON_PARSE(data),'$.active') as active,
                                        JSON_EXTRACT_SCALAR(JSON_PARSE(data),'$.selectedCondition') as selected_condition,
                                        JSON_EXTRACT(JSON_PARSE(data),'$.selectedVariable') as selected_variable,
                                        DENSE_RANK() over(partition by _id order by updated_epoch desc) as R,
                                        cast(JSON_EXTRACT(JSON_PARSE(data),'$.goals') as MAP<varchar, array(row(rules array(row("order" integer, amount integer, distance integer, index integer)), timeSlot array(row(fromTime varchar, toTime varchar)), uuid varchar))>)  as goals
                                        
                                    from hive.raw.mongodb_rapidopayroll_incentives_immutable
                                    where json_array_contains(cities, 'Delhi')
                                        and startDate between '2021-06-14' and '2021-06-20'
                                        and endDate between '2021-06-14' and '2021-06-20'
                                ) as x cross join unnest(x.goals) as y(key, value)
                            where R = 1 ) as xx cross join unnest(xx.value) as yy(rules, timeslot, uuid)
                         ) as z cross join unnest(z.rules) as zz("order", amount, distance, index)
                        )
                group by 1,2 
    ),
    daily_incentive_eligible as
                (select irs.riderid, sum(cast(i.amount as double)) as daily_incentive_eligible, min(i.incentive_eligible_orders) as daily_incentive_eligible_orders
                    from
                        (select distinct z.riderid, z.incentive_id, z.uuid as subincentive_id--, yy.paymentTime
                        from
                            (select xx.*, yy.uuid, cast(yy.rulesStatus as array(row(additionalinformation varchar, isrulecompleted varchar, isruleselected varchar, isrulestarted varchar, paymentTime varchar))) as arrr 
                            from
                                (select x.riderid, x.incentive_id, y.field, cast(y.fieldStatus as array(row(isAnyRuleSelected varchar, isSetCompleted varchar, isSetStarted varchar, rulesStatus array(row(additionalInformation varchar, isRuleCompleted varchar, isRuleSelected varchar, isRuleStarted varchar, paymentTime varchar)), uuid varchar))) as arr
                                from
                                    (select
                                        riderId as riderid,
                                        incentiveId as incentive_id,
                                        cast(incentiveProgress_json as ARRAY(row(field varchar, fieldStatus array(row(isAnyRuleSelected varchar, isSetCompleted varchar, isSetStarted varchar, rulesStatus array(row(additionalInformation varchar, isRuleCompleted varchar, isRuleSelected varchar, isRuleStarted varchar, paymentTime varchar)), uuid varchar))))) as arr

                                    from raw.mongodb_rapidopayroll_incentive_riders_snapshot
                                    where incentiveType = 'Daily'
                                        and orderdate between '2021-06-14' and '2021-06-20'
                                    ) as x cross join unnest(x.arr) as y(field, fieldStatus)
                                )as xx cross join unnest(xx.arr) as yy(isAnyRuleSelected, isSetCompleted, isSetStarted, rulesStatus, uuid)
                            ) as z cross join unnest(z.arrr) as zz(additionalinformation, isrulecompleted, isruleselected, isrulestarted, paymentTime)
                        where zz.paymentTime <> '0'
                        ) irs
                            join incentive i
                                on irs.incentive_id = i.incentive_id and irs.subincentive_id = i.subincentive_id
                    group by 1
                ),
    weekly_incentive_eligible as
                (select irs.riderid, sum(cast(i.amount as double)) as weekly_incentive_eligible, min(i.incentive_eligible_orders) as weekly_incentive_eligible_orders
                    from
                        (select distinct z.riderid, z.incentive_id, z.uuid as subincentive_id--, yy.paymentTime
                        from
                            (select xx.*, yy.uuid, cast(yy.rulesStatus as array(row(additionalinformation varchar, isrulecompleted varchar, isruleselected varchar, isrulestarted varchar, paymentTime varchar))) as arrr 
                            from
                                (select x.riderid, x.incentive_id, y.field, cast(y.fieldStatus as array(row(isAnyRuleSelected varchar, isSetCompleted varchar, isSetStarted varchar, rulesStatus array(row(additionalInformation varchar, isRuleCompleted varchar, isRuleSelected varchar, isRuleStarted varchar, paymentTime varchar)), uuid varchar))) as arr
                                from
                                    (select
                                        riderId as riderid,
                                        incentiveId as incentive_id,
                                        cast(incentiveProgress_json as ARRAY(row(field varchar, fieldStatus array(row(isAnyRuleSelected varchar, isSetCompleted varchar, isSetStarted varchar, rulesStatus array(row(additionalInformation varchar, isRuleCompleted varchar, isRuleSelected varchar, isRuleStarted varchar, paymentTime varchar)), uuid varchar))))) as arr

                                    from raw.mongodb_rapidopayroll_incentive_riders_snapshot
                                    where incentiveType = 'Weekly Fixed' 
                                    ) as x cross join unnest(x.arr) as y(field, fieldStatus)
                                )as xx cross join unnest(xx.arr) as yy(isAnyRuleSelected, isSetCompleted, isSetStarted, rulesStatus, uuid)
                            ) as z cross join unnest(z.arrr) as zz(additionalinformation, isrulecompleted, isruleselected, isrulestarted, paymentTime)
                        where zz.paymentTime <> '0'
                        ) irs
                            join incentive i
                                on irs.incentive_id = i.incentive_id and irs.subincentive_id = i.subincentive_id
                    group by 1
                ),
    daily_incentive_achieved as
                (select riderid,
                        sum(daily_incentive_achieved) as daily_incentive_achieved
                    from
                        (
                        select riderid, 
                                tincentiveIdl,
                                subIncentiveId,
                                incentivestage,
                                yyyymmdd,
                                cast(amount as double) as daily_incentive_achieved
                            from raw.mongodb_rapidopayroll_riderspaymentnew_immutable 
                            where incentivetype = 'Daily' 
                                and status = 'success' 
                                and yyyymmdd between '20210614' and '20210620'
                            group by 1,2,3,4,5,6
                        )
                    group by 1
                ),
    weekly_incentive_achieved as
                (select riderid,
                        sum(weekly_incentive_achieved) as weekly_incentive_achieved
                    from
                        (
                        select riderid, 
                                tincentiveIdl,
                                subIncentiveId,
                                incentivestage,
                                yyyymmdd,
                                cast(amount as double)as weekly_incentive_achieved
                            from raw.mongodb_rapidopayroll_riderspaymentnew_immutable 
                            where incentivetype = 'Weekly Fixed' 
                                and status = 'success' 
                                and yyyymmdd between '20210614' and '20210620'
                            group by 1,2,3,4,5,6
                        ) --as wia
                        --join incentive i
                          --      on wia.incentive_id = i.incentive_id and wia.subIncentiveId = i.subincentive_id
                    group by 1
                )
                select (case when daily.riderid is null then weekly.riderid else daily.riderid end) as riderid,
            daily.daily_incentive_eligible, 
            daily.daily_incentive_achieved, 
            daily.daily_incentive_eligible_orders,
            weekly.weekly_incentive_eligible, 
            weekly.weekly_incentive_achieved,
            weekly.weekly_incentive_eligible_orders
        from
            (
            select die.riderid, 
                die.daily_incentive_eligible, 
                dia.daily_incentive_achieved,
                die.daily_incentive_eligible_orders
            from 
                daily_incentive_eligible die 
                    left join 
                        daily_incentive_achieved dia 
                            on die.riderid = dia.riderid
            ) as daily
            full outer join
            (
            select wie.riderid, 
                wie.weekly_incentive_eligible, 
                wia.weekly_incentive_achieved,
                wie.weekly_incentive_eligible_orders
            from 
                weekly_incentive_eligible wie 
                    left join 
                        weekly_incentive_achieved wia 
                            on wie.riderid = wia.riderid
            ) as weekly
            on daily.riderid = weekly.riderid                    
               """
    
    print(incentive_eligible)

    df_incentive_eligible = pd.read_sql(incentive_eligible, presto_conn)
    
    print(" fetching incentive eligible captains", len(df_incentive_eligible))

    return df_incentive_eligible

df_incentive_eligible = pd.DataFrame()

#for day in date_list : 

incentive_eligible = get_incentive_eligible()
df_incentive_eligible = pd.concat([df_incentive_eligible,incentive_eligible])
    

df_incentive_eligible['week'] = '2021-24'
df_incentive_eligible.to_csv('incentive_eligible_achieved_Delhi.csv')

with 
    incentive as
                (select incentive_id, 
                        subincentive_id,
                        max("order") as incentive_eligible_orders,
                        sum(amount) as amount
                    from
                    (select z.R, z.incentive_id, z.uuid as subincentive_id, zz.amount, zz.index, zz."order", zz.distance
                        from
                        (
                        select  xx.R, xx.incentive_id, xx.incentive_name, xx.incentive_type, xx.city, xx.start_date, xx.end_date, xx.selected_condition, xx.selected_variable, xx.key, element_at(yy.timeslot,1).fromTime as from_time, element_at(yy.timeslot,1).toTime as to_time, yy.uuid, 
                                cast(yy.rules as array(row("order" integer, amount integer, distance integer, index integer))) as rules
                            from
                            (
                            select x.R, x.incentive_id, x.incentive_name, x.incentive_type, x.cit

In [104]:
df_incentive_eligible

,riderid,daily_incentive_eligible,daily_incentive_achieved,daily_incentive_eligible_orders,weekly_incentive_eligible,weekly_incentive_achieved,weekly_incentive_eligible_orders,week
0,5f1823e49b5aec89710df763,280.0,370.0,5,628.0,542.0,26.0,2021-22
1,5fb4cf6a90d779573ebba51c,168.0,57.0,5,NaN,NaN,NaN,2021-22
2,5d43ea6fff23f474ddf7ddcc,52.0,32.0,5,NaN,NaN,NaN,2021-22
3,5b9f28f24108bf400d5aeeac,400.0,237.0,5,380.0,265.0,38.0,2021-22
4,5de63b02ac56a62ec5e6c567,50.0,30.0,5,NaN,NaN,NaN,2021-22
...,...,...,...,...,...,...,...,...
3099,5cda7205377155163cf9a2b0,50.0,30.0,5,NaN,NaN,NaN,2021-22
3100,5f48e037be98beb130360c4d,50.0,20.0,6,NaN,NaN,NaN,2021-22
3101,604dc7861630b4553b3a36b7,100.0,24.0,10,NaN,NaN,NaN,2021-22
3102,5d1f02173b752c45cf97e6cd,200.0,125.0,19,NaN,NaN,NaN,2021-22


In [35]:
dstart =  pd.to_datetime('2021-05-31')
dend =  pd.to_datetime('2021-06-20')

date_list = [dstart + timedelta(days=x) for x in range((dend-dstart).days + 1)
        if (dstart + timedelta(days=x)).weekday() == 6]

date_list = [x.strftime('%Y-%m-%d') for x in date_list]
date_list

['2021-06-06', '2021-06-13', '2021-06-20']

In [190]:
# Get CU RF segments

def get_rf_segments(start):
    
    print("fetching RF Segments data for",start)

    orders_query = """SELECT captainid as rider, day, frequency_segment, recency_segment
                      from datasets.captain_cu_immutable
                      where day = date('{sd}')
                            -- AND city = 'Hyderabad'
                            AND segment = 'RF'                      
               """.format( sd = start)
    
#     print(orders_query)

    df_orders_data = pd.read_sql(orders_query, presto_conn)
    
    print(" fetching completed for rf segments data", len(df_orders_data))

    return df_orders_data

rf_segments = pd.DataFrame()

#for day in date_list : 

week_rf = get_rf_segments('2021-06-13')
rf_segments = pd.concat([rf_segments,week_rf])
    


rf_segments['recency_segment'] = rf_segments['recency_segment'].apply(lambda x : x.title())
rf_segments['frequency_segment'] = rf_segments['frequency_segment'].apply(lambda x : x.title())
rf_segments['Segment'] = rf_segments['recency_segment'] +'-'+ rf_segments['frequency_segment']
rf_segments['week'] = pd.to_datetime(rf_segments['day']).apply(lambda x : (x + timedelta(1)).strftime('%Y-%V'))
rf_segments

fetching RF Segments data for 2021-06-13
 fetching completed for rf segments data 178306


,rider,day,frequency_segment,recency_segment,Segment,week
0,5bb8ecb539bdcb598284603a,2021-06-13,Midperforming,Recent,Recent-Midperforming,2021-24
1,5d7f06dea24c96105e50cfb8,2021-06-13,Underperforming,Inactive,Inactive-Underperforming,2021-24
2,5c6797a7f2edc733675b6178,2021-06-13,Midperforming,Recent,Recent-Midperforming,2021-24
3,5aa62f1dda04011f73caeaa4,2021-06-13,Midperforming,Inactive,Inactive-Midperforming,2021-24
4,5fec73c3857d1b343721d0a2,2021-06-13,Midperforming,Recent,Recent-Midperforming,2021-24
...,...,...,...,...,...,...
178301,5fb20bff0a41be25868869d4,2021-06-13,Midperforming,Recent,Recent-Midperforming,2021-24
178302,5d5c2886d0286d106d5af355,2021-06-13,Midperforming,Recent,Recent-Midperforming,2021-24
178303,5d148cbb3b752c45cf8ac695,2021-06-13,Underperforming,Dormant,Dormant-Underperforming,2021-24
178304,5f9993feff48fe6b2af381f2,2021-06-13,Underperforming,Inactive,Inactive-Underperforming,2021-24


In [191]:
rf_segments = rf_segments[['rider','week','Segment']]
rf_segments.to_csv('cu_rf_segments.csv')

In [37]:
rf_segments

,rider,week,Segment
0,5e4ec1f691436136db03d38c,2021-24,Inactive-Midperforming
1,5eaf1860693f46811b8cb01f,2021-24,Recent-Midperforming
2,5dd604d1cb724727131a57cb,2021-24,Inactive-Midperforming
3,5cd66891377155163cf6f351,2021-24,Inactive-Underperforming
4,5d85dc7ff354c15d38c6f870,2021-24,Recent-Underperforming
...,...,...,...
178301,5d57b545ad6dea53c8e2dc3e,2021-24,Inactive-Underperforming
178302,5d9172e9a1f0a71c604f9281,2021-24,Recent-Midperforming
178303,5f8ab7368ab60b582e88307e,2021-24,Recent-Underperforming
178304,5cd90fafd2ee6e6cf2e5b4dc,2021-24,Recent-Underperforming


In [6]:
rf_segments['Segment'].value_counts()

Inactive-Underperforming    50831
Recent-Midperforming        49470
Inactive-Midperforming      44378
Recent-Underperforming      32483
Dormant-Midperforming       18627
Dormant-Underperforming     18233
Recent-Highperforming        2957
Inactive-Highperforming      1962
Dormant-Highperforming        680
Name: Segment, dtype: int64

In [3]:
rf_segments = pd.read_csv('cu_rf_segments.csv')
df_levels = pd.read_csv('priority_table.csv')
df_weekly_incentive = pd.read_csv('weekly_incentive_Delhi.csv')
df_daily_incentive = pd.read_csv('daily_incentive_Delhi.csv')
df_incentive_eligible = pd.read_csv('incentive_eligible_achieved_Delhi.csv')

In [135]:
#df_levels = df_levels[df_levels['week']=='2021-06']
#df_levels['week'] = '2021-07'

In [136]:
#df_final = df_levels.merge(df_weekly_incentive, how='left', left_on=['rider','week'], right_on=['riderid','week'])
#df_final = df_final.merge(df_daily_incentive, how='left', left_on=['rider','week'], right_on=['riderid','week'])
#df_final = df_final.merge(rf_segments, how='left', left_on=['rider','week'], right_on=['rider','week'])

#df_final = df_final[['rider','week','priority','Segment','weekly_incentive','weekly_incentive_order_count','weekly_ipr','daily_incentive','daily_incentive_order_count','daily_ipr']]

In [4]:
df_levels = df_levels[df_levels['week']<='2021-23']
df_levels = df_levels.sort_values(['rider','week'], ascending=False).groupby(['rider']).first().reset_index()
df_levels['week'] = '2021-24'
df_levels.head()

,rider,ltr_bucket,all_service_rides,rating,max_priority,APR,cancel,week,priority
0,5737c6bfddbec2203f73320c,1200+,3,5.0,1.0,0.785714,0.0,2021-24,7.0
1,5737c6dbddbec2203f7332fc,1200+,10,5.0,1.0,0.800000,0.0,2021-24,6.0
2,5737c714ddbec2203f7334ee,250-550,2,0.0,4.0,0.080000,0.0,2021-24,8.0
3,5737c716ddbec2203f7334fd,50-100,1,0.0,6.0,0.250000,0.0,2021-24,8.0
4,5737c718ddbec2203f733509,1200+,8,0.0,1.0,0.888889,0.0,2021-24,8.0


In [5]:
df_final = rf_segments.merge(df_levels, how='left', left_on=['rider','week'], right_on=['rider','week'])
df_final = df_final.merge(df_daily_incentive, how='left', left_on=['rider','week'], right_on=['riderid','week'])
df_final = df_final.merge(df_weekly_incentive, how='left', left_on=['rider','week'], right_on=['riderid','week'])
df_final = df_final.merge(df_incentive_eligible, how='left', left_on=['rider','week'], right_on=['riderid','week'])


df_final = df_final[['rider','week','priority','Segment','weekly_incentive','weekly_incentive_order_count','weekly_ipr',
                     'daily_incentive','daily_incentive_order_count','daily_ipr','daily_incentive_eligible','daily_incentive_achieved',
                     'weekly_incentive_eligible','weekly_incentive_achieved','daily_incentive_eligible_orders','weekly_incentive_eligible_orders']]

In [110]:
df_final.head()

,rider,week,priority,Segment,weekly_incentive,weekly_incentive_order_count,weekly_ipr,daily_incentive,daily_incentive_order_count,daily_ipr,daily_incentive_eligible,daily_incentive_achieved,weekly_incentive_eligible,weekly_incentive_achieved,daily_incentive_eligible_orders,weekly_incentive_eligible_orders
0,5ffee4ec498ff32c8bd8a75f,2021-22,8.0,Dormant-Midperforming,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5d88e1daf354c15d38c9a72f,2021-22,8.0,Inactive-Midperforming,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5cbe60b254bc7263ff484267,2021-22,8.0,Dormant-Underperforming,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5f3653b044a784fb25b004c1,2021-22,NaN,Inactive-Underperforming,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5d8b4b0bf354c15d38cc290c,2021-22,NaN,Inactive-Midperforming,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
df_final['Segment'].value_counts()

Inactive-Underperforming    59944
Inactive-Midperforming      58386
Recent-Midperforming        22123
Recent-Underperforming      17306
Dormant-Underperforming      8281
Dormant-Midperforming        7396
Inactive-Highperforming      3782
Recent-Highperforming         849
Dormant-Highperforming        239
Name: Segment, dtype: int64

In [180]:
#Fetch the city of riders week wise

df_rider_city = pd.DataFrame()

rider_city_query = """
        SELECT distinct rider,
                --CAST(DATE_TRUNC('week',CAST(orderdate AS DATE)) as varchar) as order_week,
                serviceobj_city as city
            from legacy.orders
            where rider != ''
                and orderdate >= '2021-03-01'
                and orderdate <= '2021-06-20' """

print(rider_city_query)
df_temp = pd.read_sql(rider_city_query, presto_conn)

df_rider_city = pd.concat([df_rider_city,df_temp])
print(len(df_rider_city))


        SELECT distinct rider,
                --CAST(DATE_TRUNC('week',CAST(orderdate AS DATE)) as varchar) as order_week,
                serviceobj_city as city
            from legacy.orders
            where rider != ''
                and orderdate >= '2021-03-01'
                and orderdate <= '2021-06-20' 
343915


In [181]:
#df_rider_city['week'] = pd.to_datetime(df_rider_city['order_week'])
#df_rider_city['week'] = df_rider_city['week'].apply(lambda x : x.strftime('%Y-%V'))
df_rider_city['week'] = '2021-24'
df_rider_city.head()
df_rider_city.to_csv('Rider_City.csv',index=False)

In [6]:
df_rider_city = pd.read_csv('Rider_City.csv')
df_rider_city.head(5)

,rider,city,week
0,5f16c86424d6298b43b303df,Hyderabad,2021-24
1,5f6b59629025c2e35840f30f,Hyderabad,2021-24
2,5cb81b563d65ca5e255cbabe,Jaipur,2021-24
3,5d0f86f10eed7c44be2bc031,Jaipur,2021-24
4,5a6c70059b3e77610bc4889e,Bangalore,2021-24


In [7]:
df_city = df_final.merge(df_rider_city, how='left', left_on=['rider','week'], right_on=['rider','week'])
df_city.head()

,rider,week,priority,Segment,weekly_incentive,weekly_incentive_order_count,weekly_ipr,daily_incentive,daily_incentive_order_count,daily_ipr,daily_incentive_eligible,daily_incentive_achieved,weekly_incentive_eligible,weekly_incentive_achieved,daily_incentive_eligible_orders,weekly_incentive_eligible_orders,city
0,5bb8ecb539bdcb598284603a,2021-24,6.0,Recent-Midperforming,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hyderabad
1,5d7f06dea24c96105e50cfb8,2021-24,8.0,Inactive-Underperforming,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vijayawada
2,5c6797a7f2edc733675b6178,2021-24,8.0,Recent-Midperforming,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hyderabad
3,5aa62f1dda04011f73caeaa4,2021-24,8.0,Inactive-Midperforming,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hyderabad
4,5fec73c3857d1b343721d0a2,2021-24,6.0,Recent-Midperforming,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lucknow


In [8]:
df_hyderabad = df_city[(df_city['city']=='Delhi') & (df_city['week'].isin(['2021-24']))]


In [9]:
df_rides = pd.read_csv('captain_weekly_rides_rating.csv')

df_rides['week'] = pd.to_datetime(df_rides['order_week'])
df_rides['week'] = df_rides['week'].apply(lambda x : x.strftime('%Y-%V'))
df_rides = df_rides[df_rides['week']=='2021-24']
df_rides = df_rides[['rider','week','all_service_rides']]
df_rides = df_rides.rename(columns = {'all_service_rides':'rides_completed'})
df_rides.head()


,rider,week,rides_completed
1,606ad40f9772fd8c6c67d093,2021-24,70
24,6080692df58375042ab2eb2a,2021-24,30
59,5b4affb13b04d06ad5807d88,2021-24,20
82,5bc76e497e7bc30a383c5d89,2021-24,2
95,5ac870e02b0c306960e544bb,2021-24,102


In [10]:
df_hyderabad = df_hyderabad.merge(df_rides, how = 'left', left_on=['rider','week'], right_on=['rider','week'])


In [11]:
df_hyderabad['apparent_weekly_ipr'] = df_hyderabad['weekly_incentive']/df_hyderabad['rides_completed']
df_hyderabad['apparent_daily_ipr'] = df_hyderabad['daily_incentive']/df_hyderabad['rides_completed']

In [75]:
#df_hyderabad[(df_hyderabad['Segment']=='Recent-Highperforming') & (df_hyderabad['priority']==1)]['Actual_IPR'].sort_values()

681071     1.648352
607499     2.027027
813392     2.197802
1124464    2.272727
618523     2.298851
150332     2.302632
129291     2.857143
825970     2.966102
57889      3.030303
410920     3.174603
175862          NaN
847330          NaN
969318          NaN
1071858         NaN
Name: Actual_IPR, dtype: float64

In [12]:
df_hyderabad = df_hyderabad[df_hyderabad['rides_completed'].notna()]


In [13]:
df_hyderabad.pivot_table(index='Segment', columns='priority', values='weekly_incentive_order_count', aggfunc='median')


priority,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0
Segment,,,,,,,,
Dormant-Highperforming,NaN,NaN,NaN,NaN,NaN,30.0,NaN,44.0
Dormant-Midperforming,NaN,NaN,65.0,55.0,25.0,18.0,18.0,18.0
Dormant-Underperforming,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.5
Inactive-Highperforming,30.0,85.0,75.0,60.0,63.0,50.0,48.0,31.0
Inactive-Midperforming,NaN,NaN,NaN,20.0,31.0,31.0,29.5,25.0
Inactive-Underperforming,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.0
Recent-Highperforming,87.0,83.5,45.0,45.0,55.0,67.0,35.0,42.0
Recent-Midperforming,50.0,56.0,33.0,38.0,34.0,32.0,24.0,32.0
Recent-Underperforming,NaN,NaN,NaN,NaN,30.0,24.0,23.0,24.0


In [15]:
df_hyderabad.groupby(['Segment','priority'])['rides_completed'].describe(percentiles=[0.5, 0.75, 0.85, 0.9, 0.95]).to_csv('x.csv')

In [59]:
df_hyderabad.pivot_table(index='Segment', columns='priority', values='weekly_incentive_order_count', aggfunc='median')


priority,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0
Segment,,,,,,,,
Dormant-Highperforming,NaN,NaN,NaN,NaN,NaN,30.0,NaN,44.0
Dormant-Midperforming,NaN,NaN,65.0,55.0,25.0,18.0,18.0,18.0
Dormant-Underperforming,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.5
Inactive-Highperforming,NaN,30.0,80.0,60.0,63.0,50.0,48.0,31.0
Inactive-Midperforming,NaN,NaN,NaN,20.0,31.0,31.0,30.5,27.5
Inactive-Underperforming,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.5
Recent-Highperforming,87.0,87.0,50.0,45.0,55.0,67.0,35.0,42.0
Recent-Midperforming,38.0,62.0,33.0,38.0,34.0,32.0,27.0,32.0
Recent-Underperforming,NaN,NaN,NaN,NaN,30.0,24.0,24.0,24.0


In [58]:
df_hyderabad.head()


,rider,week,priority,Segment,weekly_incentive,weekly_incentive_order_count,weekly_ipr,daily_incentive,daily_incentive_order_count,daily_ipr,daily_incentive_eligible,daily_incentive_achieved,weekly_incentive_eligible,weekly_incentive_achieved,daily_incentive_eligible_orders,weekly_incentive_eligible_orders,city,rides_completed,apparent_weekly_ipr,apparent_daily_ipr
0,5d85dc7ff354c15d38c6f870,2021-24,5.0,Recent-Underperforming,NaN,NaN,NaN,19.0,3.0,6.333333,40.0,19.0,NaN,NaN,5.0,NaN,Delhi,11.0,NaN,1.727273
1,5f4359c34980e551d8c3c233,2021-24,6.0,Dormant-Midperforming,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Delhi,NaN,NaN,NaN
2,5bc4634b7e7bc30a383badd3,2021-24,4.0,Recent-Midperforming,192.0,24.0,8.0,148.0,21.0,7.047619,183.0,148.0,323.0,192.0,5.0,34.0,Delhi,46.0,4.173913,3.217391
3,5f8155cb875b8b57be6768f8,2021-24,6.0,Recent-Midperforming,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Delhi,1.0,NaN,NaN
4,5f6440d16a7785b80b3eb7af,2021-24,8.0,Recent-Underperforming,NaN,NaN,NaN,64.0,8.0,8.000000,85.0,64.0,NaN,NaN,5.0,NaN,Delhi,18.0,NaN,3.555556


In [19]:
df_hyderabad[(df_hyderabad['priority']==8) & (df_hyderabad['rides_completed']>20)]['rider'].nunique()

378

In [304]:
df_hyderabad[(df_hyderabad['Segment']=='Recent-Highperforming') & (df_hyderabad['priority']==1) & (df_hyderabad['weekly_incentive_achieved'].notna())]


,rider,week,priority,Segment,weekly_incentive,weekly_incentive_order_count,weekly_ipr,daily_incentive,daily_incentive_order_count,daily_ipr,daily_incentive_eligible,daily_incentive_achieved,weekly_incentive_eligible,weekly_incentive_achieved,daily_incentive_eligible_orders,weekly_incentive_eligible_orders,city,rides_completed,apparent_weekly_ipr,apparent_daily_ipr
4024,5bb0c0e06c55f962e6e48032,2021-24,1.0,Recent-Highperforming,910.0,97.0,9.381443,450.0,61.0,7.377049,496.0,432.0,910.0,910.0,4.0,42.0,Delhi,109.0,8.348624,4.128440
5594,5c45c4289359a37c0db8f648,2021-24,1.0,Recent-Highperforming,703.0,72.0,9.763889,302.0,40.0,7.550000,302.0,302.0,703.0,703.0,5.0,34.0,Delhi,74.0,9.500000,4.081081
6092,5d67658f357698118397330c,2021-24,1.0,Recent-Highperforming,280.0,35.0,8.000000,266.0,39.0,6.820513,316.0,266.0,380.0,280.0,5.0,42.0,Delhi,76.0,3.684211,3.500000
12214,5a0d5d28ce3dad6b606187f9,2021-24,1.0,Recent-Highperforming,280.0,35.0,8.000000,136.0,19.0,7.157895,182.0,136.0,380.0,280.0,5.0,42.0,Delhi,41.0,6.829268,3.317073
19274,5c71032ade14193306103d9f,2021-24,1.0,Recent-Highperforming,910.0,97.0,9.381443,661.0,86.0,7.686047,671.0,605.0,910.0,910.0,4.0,42.0,Delhi,126.0,7.222222,5.246032
20094,5d9d72d361bf5a14d095f9b3,2021-24,1.0,Recent-Highperforming,770.0,87.0,8.850575,255.0,37.0,6.891892,319.0,255.0,910.0,770.0,5.0,42.0,Delhi,87.0,8.850575,2.931034
24592,5ca33ecc38f05017d5788873,2021-24,1.0,Recent-Highperforming,910.0,97.0,9.381443,393.0,55.0,7.145455,393.0,393.0,910.0,910.0,5.0,42.0,Delhi,99.0,9.191919,3.969697


In [25]:
df_levels[df_levels['rider'].isin(df_hyderabad[(df_hyderabad['Segment']=='Recent-Highperforming') & (df_hyderabad['priority']==2) & (df_hyderabad['daily_incentive'].notna())]['rider'])]

,rider,ltr_bucket,all_service_rides,rating,max_priority,APR,cancel,week,priority
6093,5a944ef4c5a70933a82835b8,800-1200,79,4.8,2,0.950980,0.041237,2021-06,2.0
15635,5b93bf0c98cab328cc094bc6,1200+,68,4.6,1,0.986486,0.000000,2021-06,2.0
26481,5c0b8eecf7dad14c3f3c5028,800-1200,70,4.6,2,0.952941,0.037037,2021-06,2.0
33705,5c36d64ad8a7ad6ca4fb8c6e,1200+,70,4.5,1,0.926316,0.056818,2021-06,2.0
37556,5c4b4f014a267149c77b57b8,800-1200,67,4.7,2,0.900000,0.077778,2021-06,2.0
38333,5c503e75976854537ab19b93,1200+,61,4.5,1,0.986301,0.041667,2021-06,2.0
42231,5c61da09f2edc7336755eb46,1200+,86,4.5,1,0.990385,0.019417,2021-06,2.0
46799,5c73d9c75e042733c9be0482,1200+,62,4.5,1,0.893617,0.011905,2021-06,2.0
61528,5cb05f83194e920db09e1448,800-1200,111,4.6,2,0.992908,0.050000,2021-06,2.0
67556,5cc820713d65ca5e2566ae3f,1200+,76,4.5,1,0.954955,0.084906,2021-06,2.0
